# Using NHGIS Data with Python

For this demonstration we will use data obtained from the [National Historical Geographic Information System (NHGIS)](https://www.nhgis.org/), which is housed within [IPUMS](https://www.ipums.org/) at the University of Minnesota.

NHGIS provides both cross-sectional and longitudinal aggregate data from historical and current censuses, as well as geographic boundary (GIS) files that can be used with these tabular data products.

It's possible to obtain data directly from NHGIS using its [extract engine](https://data2.nhgis.org/main), where users can specify details about the data they'd like to download. However, IPUMS also provides an [API interface](https://developer.ipums.org/docs/apiprogram/) for several of its products, including NHGIS. This means that we can request data from NHGIS in a reproducible and documentable Python workflow.

In [ ]:
!pip install ipumspy

## Python setup

First, we'll load the necessary libraries and moduls for the demonstration. Much of this demo is modeled off of the `ipumspy` library, which already provides some tools to interact with the IPUMS API in Python.

In [ ]:
from ipumspy import IpumsApiClient, BaseExtract, readers
from zipfile import ZipFile
from pathlib import Path
from typing import Any, Dict, List, Optional, Type, Union, Tuple

import pandas as pd
import geopandas as gpd
import re
import requests
import os
import getpass

In [ ]:
%run nhgis_api_workflow_funs.py

## IPUMS API setup

NHGIS data is delivered in what is called an _extract_ or _data extract_. To request data, we first must build an _extract request_, which specifies the data parameters we want included in our ultimate extract.

To get started with the IPUMS API for NHGIS, you'll have to [register](https://uma.pop.umn.edu/nhgis/registration/new) (for free) as an NHGIS user. You'll also have to request an API key by following the instructions [here](https://account.ipums.org/api_keys).

Once you've got your key, you can initialize an API client to facilitate interactions with the IPUMS API. You can do this by initializing an `NhgisApiClient` object with your API key. 

In our case, we store our key as the `IPUMS_API_KEY` environment variable to make sure it's kept secret. Run the following code and put in your own API key in the promt to store your key in your environment:
(Note: If you forget, you can get your generated key [here](https://account.ipums.org/api_keys))

In [ ]:
os.environ["IPUMS_API_KEY"] = getpass.getpass("Enter your API Key")

Now, we can get our key by referencing the `IPUMS_API_KEY` environment variable:

In [ ]:
nhgis_client = NhgisApiClient(api_key = os.environ.get("IPUMS_API_KEY"))
assert nhgis_client.api_key is not None, 'IPUMS API KEY is not set. Please make sure to enter your key in the prompt after running the previous cell'

## Obtaining data with an NHGIS extract request

Now, we need to specify the parameters that identify the data we'd like to obtain from NHGIS.

NHGIS data is organized into [datasets](https://www.nhgis.org/overview-nhgis-datasets), [time series tables](https://www.nhgis.org/time-series-tables), and [shapefiles](https://www.nhgis.org/gis-files).

Datasets are composed of many data tables, which contain tabulated aggregate data for a specific metric (for instance, counts of persons by sex and age). Additionally, each dataset is available at multiple geographic levels. Therefore, when we specify a dataset, we must also indicate which tables and geographic levels we want to obtain for that dataset.

Time series tables contain longitudinal data that has been standardized over time. These also are available at multiple geographic levels, and therefore require a geographic level specification.

Shapefiles represent geographic boundary files for a particular year and geographic level. Shapefiles can be easily specified with their name.

For instance, a sample extract definition might look something like this:

In [ ]:
extract = NhgisExtract(
    description = "Sample extract for python demo",
    datasets = {
        "1990_STF1": {
            "dataTables": ["NP1"],
            "geogLevels": ["county"]
        }
    },
    time_series_tables = {
        "A00": {
            "geogLevels": ["state"]
        }
    },
    shapefiles = ["us_state_1790_tl2000"],
    data_format = "csv_no_header"
)

This `NhgisExtract` object contains an extract request for one table at the county level from the `1990_STF1` dataset, a full time series table (`A00`) at the state level, and a single state-level shapefile for geographic boundaries in 1790.

Of course, it's not expected that you'd know all the NHGIS data source codes off the top of your head. Fortunately, you can also access NHGIS metadata through the API for help finding the data you want to include in your request.

## NHGIS metadata

The IPUMS API provides both detailed and summary metadata about available NHGIS data sources. We can request this metadata using the `nhgis_metadata()` method provided by `NhgisApiClient`.

To retrieve summary metadata for datasets, time series tables, or shapefiles, use the `type` argument. This returns the metadata formatted as JSON, but this can be converted to a more familiar tabular data structure using the `pandas` library.

For instance, to see all available datasets:

In [ ]:
meta = nhgis_client.nhgis_metadata(type = "datasets")
pd.json_normalize(meta)

To obtain detailed metadata about a particular data source, pass its name to the corresponding argument of `nhgis_metadata()`.

In [ ]:
meta = nhgis_client.nhgis_metadata(dataset = "2015_2019_ACS5a")
tables = pd.json_normalize(meta["dataTables"])

tables

We can use `pandas` to easily filter through NHGIS metadata to identify particular data sources of interest:

In [ ]:
tables.loc[tables.description.str.contains("Median Age")]

From this, we can identify the data we want to include in our extract.

## Downloading NHGIS data

Once we've prepared our extract definition, we can submit it for processing:

In [ ]:
nhgis_client.submit_extract(extract)

# We notice that the extract now has entered the processing queue:
nhgis_client.extract_status(extract)

It may take a bit for the extract to be processed, but we can wait for its completion and then download the files:

In [ ]:
# Ensure extract has completed before attempting download
nhgis_client.wait_for_extract(extract)

nhgis_client.download_nhgis_extract(extract)

## Read downloaded files

NHGIS files are shipped in zipped format, often with multiple files contained in the provided zip file. Some python libraries can handle reading through zip archives, but not typically if multiple files are contained within the archive.

We've defined a few helper functions to handle these structures more easily, allowing us to use the `file_select` argument to indicate a specific file to load from within a zip archive. This way, we can load a file with Python without having to extract all of its contents.

First, let's take a look at the files that are contained in our downloaded extract:

In [ ]:
# NHGIS file names include the extract id number, which we can obtain from the extract object:
filename = 'nhgis{0}_csv.zip'.format(str(extract.extract_id).zfill(4))

# Peek at contents of this zip archive
nhgis_list_files(filename)

There are both .txt and .csv files in our resulting extract. The .txt files are codebooks that tell us how to interpret the columns of the data in the corresponding .csv file. However, these are designed to be human readable, not machine readable. To load our data, we'll want to restrict our focus just to the .csv files:

In [ ]:
nhgis_list_files(filename, pattern = "csv$")

We can use our helpers to read one of the files without having to unzip:

In [ ]:
read_nhgis(filename, file_select = "ds120_1990_county.csv")

An extract's shapefiles are shipped in a separate zip archive from its data files. We can load spatial data with the geopandas library, which we have wrapped into our helper function `read_nhgis_shp()`. In this case, there's only one file within the zip archive, so we don't need to specify a particular file to load:

In [ ]:
# Shapefile zip archive name
filename = 'nhgis{0}_shape.zip'.format(str(extract.extract_id).zfill(4))
shp = read_nhgis_shp(filename)

We can see that our boundaries represent 1790 state boundaries, just as was requested in our original extract definition:

In [ ]:
shp.plot()

And just like that, we've gone from a completly new extract request to NHGIS data at our fingertips! 

To obtain different data, all that's required is to create a new extract request. Then, the same pipeline can be followed to load data into Python.